In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf 

<b>Задание 1.</b> Загрузите данные, проверьте типы, пропущенные значения.  
Сколько пропущенных значений встретилось в датасете?

In [2]:
df = pd.read_csv("https://stepik.org/media/attachments/lesson/387691/cars.csv")

In [3]:
df.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [4]:
df.dtypes

car_ID                int64
symboling             int64
CarName              object
fueltype             object
aspiration           object
doornumber           object
carbody              object
drivewheel           object
enginelocation       object
wheelbase           float64
carlength           float64
carwidth            float64
carheight           float64
curbweight            int64
enginetype           object
cylindernumber       object
enginesize            int64
fuelsystem           object
boreratio           float64
stroke              float64
compressionratio    float64
horsepower            int64
peakrpm               int64
citympg               int64
highwaympg            int64
price               float64
dtype: object

In [5]:
df.isna().sum()

car_ID              0
symboling           0
CarName             0
fueltype            0
aspiration          0
doornumber          0
carbody             0
drivewheel          0
enginelocation      0
wheelbase           0
carlength           0
carwidth            0
carheight           0
curbweight          0
enginetype          0
cylindernumber      0
enginesize          0
fuelsystem          0
boreratio           0
stroke              0
compressionratio    0
horsepower          0
peakrpm             0
citympg             0
highwaympg          0
price               0
dtype: int64

<b>Задание 2.</b> Сгенерируем первый признак.  

Использовать полное название машины – не самый хороший вариант, поэтому создадим новый признак – марку автомобиля (company). Для этого используйте столбец CarName, разбейте значения ячеек по пробелу и запишите в колонку первый элемент. Например:  

<b>'audi 100 ls' → 'audi'</b>  

Сколько всего уникальных марок машины встречается в датасете? Столбец CarName с полным названием машины удалите из датасета, а также car_ID, они не пригодятся для дальнейшего анализа.

In [6]:
df["company"] = df.CarName.apply(lambda x : x.split(" ")[0])
df.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price,company
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0,alfa-romero
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0,alfa-romero
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0,alfa-romero
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0,audi
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0,audi


In [7]:
df.company.nunique()

28

In [8]:
df = df.drop(["CarName", "car_ID"], 1)

<b>Задание 3.</b> Теперь внимательнее посмотрите на уникальные значения company. Часть из них оказалась с ошибками!  
- 'maxda' → 'mazda'
- 'Nissan' → 'nissan'
- 'porcshce' → 'porsche'
- 'toyouta' → 'toyota'
- 'vokswagen' & 'vw' → 'volkswagen'  

Сколько уникальных производителей осталось в итоге?

In [9]:
df["company"] = df.company.replace("maxda", "mazda")
df["company"] = df.company.replace("Nissan", "missan")
df["company"] = df.company.replace("missan", "nissan")
df["company"] = df.company.replace("porcshce", "porsche")
df["company"] = df.company.replace("toyouta", "toyota")
df["company"] = df.company.replace("vokswagen", "volkswagen")
df["company"] = df.company.replace("vw", "volkswagen")

In [10]:
df.company.nunique()

22

<b>Задание 4.</b> Отлично! Чтобы не перегружать модель большим количеством предикторов, оставим только часть из них также не забыв про то, что мы предсказываем – 'price'.  

После этого посчитайте корреляцию между price и другими переменными. Чему равна корреляция между price и horsepower?

In [11]:
df = df[["price", "company", "fueltype", 'aspiration','carbody', 'drivewheel', 'wheelbase', 'carlength','carwidth', 'curbweight', 'enginetype', 'cylindernumber', 'enginesize', 'boreratio','horsepower']]

In [12]:
df

,price,company,fueltype,aspiration,carbody,drivewheel,wheelbase,carlength,carwidth,curbweight,enginetype,cylindernumber,enginesize,boreratio,horsepower
0,13495.0,alfa-romero,gas,std,convertible,rwd,88.6,168.8,64.1,2548,dohc,four,130,3.47,111
1,16500.0,alfa-romero,gas,std,convertible,rwd,88.6,168.8,64.1,2548,dohc,four,130,3.47,111
2,16500.0,alfa-romero,gas,std,hatchback,rwd,94.5,171.2,65.5,2823,ohcv,six,152,2.68,154
3,13950.0,audi,gas,std,sedan,fwd,99.8,176.6,66.2,2337,ohc,four,109,3.19,102
4,17450.0,audi,gas,std,sedan,4wd,99.4,176.6,66.4,2824,ohc,five,136,3.19,115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,16845.0,volvo,gas,std,sedan,rwd,109.1,188.8,68.9,2952,ohc,four,141,3.78,114
201,19045.0,volvo,gas,turbo,sedan,rwd,109.1,188.8,68.8,3049,ohc,four,141,3.78,160
202,21485.0,volvo,gas,std,sedan,rwd,109.1,188.8,68.9,3012,ohcv,six,173,3.58,134
203,22470.0,volvo,diesel,turbo,sedan,rwd,109.1,188.8,68.9,3217,ohc,six,145,3.01,106


In [13]:
df.corr()

,price,wheelbase,carlength,carwidth,curbweight,enginesize,boreratio,horsepower
price,1.000000,0.577816,0.682920,0.759325,0.835305,0.874145,0.553173,0.808139
wheelbase,0.577816,1.000000,0.874587,0.795144,0.776386,0.569329,0.488750,0.353294
carlength,0.682920,0.874587,1.000000,0.841118,0.877728,0.683360,0.606454,0.552623
carwidth,0.759325,0.795144,0.841118,1.000000,0.867032,0.735433,0.559150,0.640732
curbweight,0.835305,0.776386,0.877728,0.867032,1.000000,0.850594,0.648480,0.750739
enginesize,0.874145,0.569329,0.683360,0.735433,0.850594,1.000000,0.583774,0.809769
boreratio,0.553173,0.488750,0.606454,0.559150,0.648480,0.583774,1.000000,0.573677
horsepower,0.808139,0.353294,0.552623,0.640732,0.750739,0.809769,0.573677,1.000000


<b>Задание 5.</b> Последний шаг в подготовке данных: линейная регрессия в python не справляется с категориальными переменными (типом object в pandas), поэтому давайте применим pd.get_dummies().

In [14]:
df.dtypes

price             float64
company            object
fueltype           object
aspiration         object
carbody            object
drivewheel         object
wheelbase         float64
carlength         float64
carwidth          float64
curbweight          int64
enginetype         object
cylindernumber     object
enginesize          int64
boreratio         float64
horsepower          int64
dtype: object

In [15]:
tmp_df = df[["price", "wheelbase", "carlength", "carwidth", "curbweight", "enginesize", "boreratio", "horsepower"]]
df_dummy = pd.get_dummies(data=df[["company", "fueltype", "aspiration", "carbody", "drivewheel", "enginetype", "cylindernumber"]], drop_first = True)

In [16]:
new_df = pd.concat([tmp_df, df_dummy], axis=1)

In [17]:
new_df.shape

(205, 49)

In [18]:
new_df.head()

,price,wheelbase,carlength,carwidth,curbweight,enginesize,boreratio,horsepower,company_audi,company_bmw,...,enginetype_ohc,enginetype_ohcf,enginetype_ohcv,enginetype_rotor,cylindernumber_five,cylindernumber_four,cylindernumber_six,cylindernumber_three,cylindernumber_twelve,cylindernumber_two
0,13495.0,88.6,168.8,64.1,2548,130,3.47,111,0,0,...,0,0,0,0,0,1,0,0,0,0
1,16500.0,88.6,168.8,64.1,2548,130,3.47,111,0,0,...,0,0,0,0,0,1,0,0,0,0
2,16500.0,94.5,171.2,65.5,2823,152,2.68,154,0,0,...,0,0,1,0,0,0,1,0,0,0
3,13950.0,99.8,176.6,66.2,2337,109,3.19,102,1,0,...,1,0,0,0,0,1,0,0,0,0
4,17450.0,99.4,176.6,66.4,2824,136,3.19,115,1,0,...,1,0,0,0,1,0,0,0,0,0


<b>Задание 6.</b> Сначала построим небольшую модель всего с одним предиктором цены (price) – horsepower.  
Какой процент изменчивости объясняет полученная модель? (округлите до целого)

In [19]:
results = smf.ols('price ~ horsepower', new_df).fit()
print(results.summary()) 
# Ответ: R-squared = 65%

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.653
Model:                            OLS   Adj. R-squared:                  0.651
Method:                 Least Squares   F-statistic:                     382.2
Date:                Thu, 02 Jun 2022   Prob (F-statistic):           1.48e-48
Time:                        13:13:06   Log-Likelihood:                -2024.0
No. Observations:                 205   AIC:                             4052.
Df Residuals:                     203   BIC:                             4059.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -3721.7615    929.849     -4.003      0.0

<b>Задание 7.</b> Теперь – две модели:  
- модель со всеми предикторами
- модель со всеми предикторами, кроме марок машин  

Обратите внимание на изменения в $R^{2}$, коэффициентах и их значимости. Какую модель лучше оставить? 



In [20]:
my_string = ""
for i in new_df.columns:
    my_string += i + ", "
my_string

'price, wheelbase, carlength, carwidth, curbweight, enginesize, boreratio, horsepower, company_audi, company_bmw, company_buick, company_chevrolet, company_dodge, company_honda, company_isuzu, company_jaguar, company_mazda, company_mercury, company_mitsubishi, company_nissan, company_peugeot, company_plymouth, company_porsche, company_renault, company_saab, company_subaru, company_toyota, company_volkswagen, company_volvo, fueltype_gas, aspiration_turbo, carbody_hardtop, carbody_hatchback, carbody_sedan, carbody_wagon, drivewheel_fwd, drivewheel_rwd, enginetype_dohcv, enginetype_l, enginetype_ohc, enginetype_ohcf, enginetype_ohcv, enginetype_rotor, cylindernumber_five, cylindernumber_four, cylindernumber_six, cylindernumber_three, cylindernumber_twelve, cylindernumber_two, '

In [41]:
X = sm.add_constant(np.array(new_df.drop("price", 1)))
model = sm.OLS(np.array(new_df.price), X)  # говорим модели, что у нас ЗП, а что НП
results = model.fit()  # строим регрессионную прямую
print(results.summary())  # смотрим результат

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.959
Model:                            OLS   Adj. R-squared:                  0.948
Method:                 Least Squares   F-statistic:                     81.09
Date:                Thu, 02 Jun 2022   Prob (F-statistic):           4.86e-89
Time:                        13:36:58   Log-Likelihood:                -1804.2
No. Observations:                 205   AIC:                             3702.
Df Residuals:                     158   BIC:                             3858.
Df Model:                          46                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -3.472e+04   1.17e+04     -2.959      0.0

In [42]:
new_df.columns

Index(['price', 'wheelbase', 'carlength', 'carwidth', 'curbweight',
       'enginesize', 'boreratio', 'horsepower', 'company_audi', 'company_bmw',
       'company_buick', 'company_chevrolet', 'company_dodge', 'company_honda',
       'company_isuzu', 'company_jaguar', 'company_mazda', 'company_mercury',
       'company_mitsubishi', 'company_nissan', 'company_peugeot',
       'company_plymouth', 'company_porsche', 'company_renault',
       'company_saab', 'company_subaru', 'company_toyota',
       'company_volkswagen', 'company_volvo', 'fueltype_gas',
       'aspiration_turbo', 'carbody_hardtop', 'carbody_hatchback',
       'carbody_sedan', 'carbody_wagon', 'drivewheel_fwd', 'drivewheel_rwd',
       'enginetype_dohcv', 'enginetype_l', 'enginetype_ohc', 'enginetype_ohcf',
       'enginetype_ohcv', 'enginetype_rotor', 'cylindernumber_five',
       'cylindernumber_four', 'cylindernumber_six', 'cylindernumber_three',
       'cylindernumber_twelve', 'cylindernumber_two'],
      dtype='object'

In [46]:
new_df2 = new_df.drop(['price', 'company_audi', 'company_bmw',
       'company_buick', 'company_chevrolet', 'company_dodge', 'company_honda',
       'company_isuzu', 'company_jaguar', 'company_mazda', 'company_mercury',
       'company_mitsubishi', 'company_nissan', 'company_peugeot',
       'company_plymouth', 'company_porsche', 'company_renault',
       'company_saab', 'company_subaru', 'company_toyota',
       'company_volkswagen', 'company_volvo'], 1)
X = sm.add_constant(np.array(new_df2))
model = sm.OLS(np.array(new_df.price), X)  # говорим модели, что у нас ЗП, а что НП
results = model.fit()  # строим регрессионную прямую
print(results.summary())  # смотрим результат

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.914
Model:                            OLS   Adj. R-squared:                  0.901
Method:                 Least Squares   F-statistic:                     72.32
Date:                Thu, 02 Jun 2022   Prob (F-statistic):           9.86e-81
Time:                        13:40:08   Log-Likelihood:                -1881.6
No. Observations:                 205   AIC:                             3817.
Df Residuals:                     178   BIC:                             3907.
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -1.7e+04    1.3e+04     -1.309      0.1

<b>Задание 8.</b> Выбранная модель объясняет примерно 90% дисперсии (окр. до целого). Среди предикторов 
10 из 27 оказались не значимыми (p > 0.05). Пример интерпретации: при единичном изменении показателя horsepower, цена 
ВОЗРАСТАЕТ на 86.8164 (без округления).